In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import shap
import matplotlib.pyplot as plt

from sklearn.model_selection import GroupShuffleSplit
from xgboost import XGBClassifier

# from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from imblearn.over_sampling.base import BaseOverSampler
from imblearn.under_sampling import TomekLinks, RandomUnderSampler, EditedNearestNeighbours, RepeatedEditedNearestNeighbours
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import make_scorer, precision_score, recall_score, classification_report
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

from sklearn.model_selection import LearningCurveDisplay, learning_curve
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import f1_score, make_scorer

from scipy import stats

import math
from sklearn.cluster import AgglomerativeClustering



# CSF EDITS
import anndata as ad
import scanpy as sc
import pandas as pd
import seaborn as sns

In [2]:
os.getcwd()

'/app/store/France_CSF/B'

In [3]:
dataset_name = "bcells_csf"

<h2>Preparazione dei dati</h2>

In [5]:
dataset = sc.read_h5ad(f"{dataset_name}.h5ad")
print(dataset.X.shape)

(2143, 17820)


In [6]:
sampleID = dataset.obs['sample']
indicator = dataset.obs['condition']
datasetID = dataset.obs['dataset']

<h2>Analisi di correlazione</h2>

In [6]:
current = dataset.copy()
data = pd.DataFrame(current.X, columns=current.var_names)
dataCorr = data.corr()

joblib.dump(dataCorr, f'{dataset_name}_dataCorr.pkl')

['bcells_csf_dataCorr.pkl']

In [7]:
dataCorr = joblib.load(f'{dataset_name}_dataCorr.pkl')
dataCorr.index = dataset.var_names
dataCorr.columns = dataset.var_names

abs_corr = dataCorr.abs()

clusters = []

for gene_i in dataCorr.index:
    # Get all correlations >= 0.9 for this gene
    high_corr_mask = abs_corr.loc[gene_i] >= 0.9
    cluster = dataCorr.columns[high_corr_mask].tolist()
    
    if len(cluster) > 1:  # Only add if cluster has more than 1 gene
        clusters.append(cluster)

joblib.dump(clusters, f"{dataset_name}_clusters.pkl")

['bcells_csf_clusters.pkl']

In [8]:
clusters = joblib.load(f"{dataset_name}_clusters.pkl")

gene_list = list(dataset.var_names)

for cluster in clusters:
    for gene in cluster:
        if gene in gene_list:
            gene_list.remove(gene)

In [9]:
current = dataset.copy()
data = pd.DataFrame(current.X, columns=current.var_names)

# Calculate variances
all_variances = {var: float(data[var].var()) for var in data.columns}

# Sort all variables by variance (highest to lowest) once
sorted_vars = sorted(all_variances.items(), key=lambda item: item[1], reverse=True)
sorted_vars_list = [var for var, _ in sorted_vars]

representative = {}
# Process clusters by size (largest first)
for cluster in sorted(clusters, key=len, reverse=True):
    cluster_tuple = tuple(cluster)

    # Find the highest variance variable in this cluster that isn't already used
    found_representative = False
    for var in sorted_vars_list:
        if var in cluster and var not in representative.values():
            representative[cluster_tuple] = var
            found_representative = True
            break

    # If we couldn't find an unused representative, use the highest variance one anyway
    if not found_representative:
        for var in sorted_vars_list:
            if var in cluster:
                representative[cluster_tuple] = var
                break

# Filter out subsets more efficiently
unique_clusters = {}
cluster_tuples = list(representative.keys())

# Sort by length once
cluster_tuples.sort(key=len)

# Use a more efficient algorithm to filter subsets
for i, cluster in enumerate(cluster_tuples):
    is_unique = True
    cluster_set = set(cluster)

    # Only check against larger clusters
    for j in range(i + 1, len(cluster_tuples)):
        larger_cluster = cluster_tuples[j]
        if cluster_set.issubset(set(larger_cluster)):
            is_unique = False
            break

    if is_unique:
        unique_clusters[cluster] = representative[cluster]


In [10]:
gene_list.extend(gene for gene in unique_clusters.values() if gene not in gene_list)

print(len(gene_list))

15651


In [11]:
datasetDeclustered = dataset[:, gene_list]
datasetDeclustered.write_h5ad(f"{dataset_name}_datasetDeclustered.h5ad")
datasetDeclustered

View of AnnData object with n_obs × n_vars = 2143 × 15651
    obs: 'condition', 'sample', 'tissue', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_genes', 'n_counts', 'predicted_labels', 'over_clustering', 'majority_voting', 'dataset', 'generalized_celltype', '_scvi_batch', '_scvi_labels', 'concat_batch'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'dataset_colors', 'generalized_celltype_colors', 'majority_voting_colors', 'neighbors', 'pca', 'sample_colors', 'umap'
    obsm: 'X_pca', 'X_umap', 'corrected_latent', 'latent'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

<h2>Divisione in train e test</h2>

In [38]:
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
from collections import defaultdict

def find_best_random_state(dataset, n_trials=1000, target_train_size=0.75, target_ratio=1.0):
    """
    Trova il miglior random_state per GroupShuffleSplit che:
    1. Si avvicina a una divisione 80% train, 20% test
    2. Ha il rapporto più bilanciato possibile (vicino a 1:1) tra le condizioni nel train e nel test
    
    Args:
        dataset: AnnData object con i dati e le annotazioni
        n_trials: numero di random_state da provare
        target_train_size: dimensione desiderata per il train set (default 0.8)
        target_ratio: rapporto desiderato tra le condizioni (default 1.0)
    
    Returns:
        Il miglior random_state trovato e le statistiche corrispondenti
    """
    best_score = float('inf')
    best_random_state = None
    best_stats = None
    
    # Conta il numero totale di campioni per condizione
    total_ms = np.sum(dataset.obs['condition'] == 'MS')
    total_ctrl = np.sum(dataset.obs['condition'] == 'CTRL')
    total_samples = len(dataset.obs)
    
    for random_state in range(n_trials):
        # Esegui la divisione
        splitter = GroupShuffleSplit(n_splits=2, test_size=1-target_train_size, random_state=random_state)
        split = splitter.split(dataset.X, groups=dataset.obs['sample'])
        train_inds, test_inds = next(split)
        
        # Crea i subset
        train = dataset[train_inds].copy()
        test = dataset[test_inds].copy()
        
        # Calcola le statistiche
        train_ms = np.sum(train.obs['condition'] == 'MS')
        train_ctrl = np.sum(train.obs['condition'] == 'CTRL')
        test_ms = np.sum(test.obs['condition'] == 'MS')
        test_ctrl = np.sum(test.obs['condition'] == 'CTRL')
        
        # Calcola i rapporti
        train_ratio = train_ms / train_ctrl if train_ctrl != 0 else float('inf')
        test_ratio = test_ms / test_ctrl if test_ctrl != 0 else float('inf')
        
        # Calcola gli scostamenti dal target
        train_size_diff = abs(len(train_inds)/total_samples - target_train_size)
        train_ratio_diff = abs(train_ratio - target_ratio)
        test_ratio_diff = abs(test_ratio - target_ratio)
        
        # Ponderazione degli scostamenti (puoi modificare questi pesi in base alle tue priorità)
        score = 0.3 * train_ratio_diff + 0.7 * test_ratio_diff + train_size_diff
        
        # Aggiorna il miglior risultato
        if score < best_score:
            best_score = score
            best_random_state = random_state
            best_stats = {
                'train_size': len(train_inds),
                'test_size': len(test_inds),
                'train_ms': train_ms,
                'train_ctrl': train_ctrl,
                'train_ratio': train_ratio,
                'test_ms': test_ms,
                'test_ctrl': test_ctrl,
                'test_ratio': test_ratio
            }
    
    return best_random_state, best_stats


best_rs, stats = find_best_random_state(dataset)
print(f"Miglior random_state: {best_rs}")
print(f"Statistiche: {stats}")

Miglior random_state: 523
Statistiche: {'train_size': 1687, 'test_size': 456, 'train_ms': 927, 'train_ctrl': 760, 'train_ratio': 1.2197368421052632, 'test_ms': 228, 'test_ctrl': 228, 'test_ratio': 1.0}


In [7]:
dataset = sc.read_h5ad(f"{dataset_name}_datasetDeclustered.h5ad")

current = pd.DataFrame(dataset.X, columns= dataset.var_names)
current.insert(0, "SampleID", sampleID.values)
current.insert(1, "Label", indicator.values)

grouped = current.groupby(['SampleID', 'Label']).size()
result_dict = {k:v for k,v in grouped.to_dict().items() if v != 0}
result_dict

{('19270', 'MS'): 217,
 ('32190', 'CTRL'): 13,
 ('41540', 'CTRL'): 177,
 ('45044', 'CTRL'): 570,
 ('49131', 'MS'): 80,
 ('58637', 'MS'): 40,
 ('60249', 'MS'): 493,
 ('71658', 'MS'): 78,
 ('74594', 'MS'): 12,
 ('83775', 'CTRL'): 228,
 ('JYJ_CSF', 'MS'): 19,
 ('KJS_CSF', 'MS'): 171,
 ('YYW_CSF', 'MS'): 45}

In [8]:
# Load the dataset
dataset = ad.read_h5ad(f"{dataset_name}_datasetDeclustered.h5ad")
current = pd.DataFrame(dataset.X, columns= dataset.var_names)
current.insert(0, "SampleID", dataset.obs['sample'].values)
current.insert(1, "Label", dataset.obs['condition'].values)

test_ids = ['83775', '49131', '71658', 'YYW_CSF']
train_ids = list(set(current['SampleID'].unique()) - set(test_ids))

print(train_ids, test_ids)

train = current[current['SampleID'].isin(train_ids)].sample(frac=1, random_state=42)
test = current[current['SampleID'].isin(test_ids)].sample(frac=1, random_state=42)

# Split the dataset using GroupShuffleSplit
# splitter = GroupShuffleSplit(n_splits=2, test_size=0.25, random_state=523)
# split = splitter.split(dataset.X, groups=dataset.obs['sample'])
# train_inds, test_inds = next(split)

# # Create AnnData subsets for training and testing
# train = dataset[train_inds].copy()
# test = dataset[test_inds].copy()

print("Dataset di train:")
print(train.shape)
print("I malati sono: ", sum(train['Label'] == 'MS'))
print("I sani sono: ", sum(train['Label'] == 'CTRL'))

print("\nDataset di test:")
print(test.shape)
print("I malati sono: ", sum(test['Label'] == 'MS'))
print("I sani sono: ", sum(test['Label'] == 'CTRL'))

y_train = train['Label'] == "MS"
x_train = train.drop(columns=['SampleID', 'Label'])

y_test = test['Label'] == "MS"
x_test = test.drop(columns=['SampleID', 'Label'])

sampleSum = dataset.X.shape[0]

print()
print(len(y_train)*100/sampleSum, len(y_test)*100/sampleSum)


['74594', '41540', 'KJS_CSF', '45044', '58637', '19270', 'JYJ_CSF', '32190', '60249'] ['83775', '49131', '71658', 'YYW_CSF']
Dataset di train:
(1712, 15653)
I malati sono:  952
I sani sono:  760

Dataset di test:
(431, 15653)
I malati sono:  203
I sani sono:  228

79.88800746616892 20.111992533831078


<h1>Addestramento modello</h1>

In [9]:
def prettyPrint(model, name, test=False):
    print(name + ":")
    print("Iperparametri: ", model.best_params_)
    print("Train f1: ", model.score(x_train, y_train))
    print("Mean f1 cross-validated: ", model.best_score_)
    best_index = model.best_index_
    print("\t\t precision \t\t recall \t\t f1-score")
    print(f"0 \t\t {model.cv_results_['mean_test_precision 0'][best_index]:.2f} \t\t\t {model.cv_results_['mean_test_recall 0'][best_index]:.2f}") 
    print(f"1 \t\t {model.cv_results_['mean_test_precision 1'][best_index]:.2f} \t\t\t {model.cv_results_['mean_test_recall 1'][best_index]:.2f}")
    print(f"Accuracy \t\t\t\t\t\t\t {model.cv_results_['mean_test_Accuracy'][best_index]:.2f}")
    print(f"macro avg \t {(model.cv_results_['mean_test_precision 0'][best_index] + model.cv_results_['mean_test_precision 1'][best_index]) / 2:.2f} \t\t\t {(model.cv_results_['mean_test_recall 0'][best_index] + model.cv_results_['mean_test_recall 1'][best_index])/2:.2f} \t\t\t {model.cv_results_['mean_test_f1'][best_index]:.2f}")
    if test:  
        print("\nTest f1 score: ", model.score(x_test, y_test))
        print(classification_report(y_test, model.predict(x_test)), "\n\n")

def bayesianOpt(pipeline, hyperparameters, iteration, scorer, njobs, x_train, y_train):
    bayesianSearchResult = BayesSearchCV(estimator = pipeline, search_spaces=hyperparameters, cv=5, n_iter=iteration, return_train_score=True,  refit='f1', scoring=scorer, n_jobs=njobs, verbose=2)
    bayesianSearchResult.fit(x_train, y_train)
    print("Iperparametri:", bayesianSearchResult.best_params_)
    return bayesianSearchResult

In [10]:
pipeline = Pipeline(steps=[
    ('scaling', MinMaxScaler()),
    ('classifier', XGBClassifier(random_state=42))])

def precision_class_0(y_true, y_pred):
    return precision_score(y_true, y_pred, average=None)[0]

def precision_class_1(y_true, y_pred):
    return precision_score(y_true, y_pred, average=None)[1]

def recall_class_0(y_true, y_pred):
    return recall_score(y_true, y_pred, average=None)[0]

def recall_class_1(y_true, y_pred):
    return recall_score(y_true, y_pred, average=None)[1]

scorer = {
    'Accuracy': 'accuracy',
    'precision 0': make_scorer(precision_class_0),
    'precision 1': make_scorer(precision_class_1),
    'recall 0': make_scorer(recall_class_0),
    'recall 1': make_scorer(recall_class_1),
    'f1': make_scorer(f1_score, average='macro')
}

<h3>Bayesian Hyperparameter Optimization</h3>

In [ ]:
param_dist = {
    'classifier__n_estimators': Integer(50, 600),  # Numero di alberi
    'classifier__max_depth': Integer(2, 15),  # Profondità dell'albero
    'classifier__learning_rate': Real(0.001, 0.7, prior='log-uniform'),  # Tasso di apprendimento
    'classifier__gamma': Real(0.0001, 1, prior='log-uniform'),  # Minimum loss reduction
    'classifier__min_child_weight': Integer(1, 8), 
    'classifier__scale_pos_weight': Categorical([1, 760/952]),
    'classifier__reg_alpha': Real(0.0001, 100, prior='log-uniform'),  # Regolarizzazione L1
    'classifier__reg_lambda': Real(0.0001, 100, prior='log-uniform')  # Regolarizzazione L2
}

bayesianOptResult = bayesianOpt(pipeline, param_dist, 600, scorer, 10, x_train, y_train)
joblib.dump(bayesianOptResult, "model1_CSF_B.pkl")

Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [13]:
bayesianOptResult = joblib.load("model1_CSF_B.pkl")
prettyPrint(bayesianOptResult, "Bayesian Hyperparameter", True)

Bayesian Hyperparameter:
Iperparametri:  OrderedDict([('classifier__gamma', 0.0001807075116973858), ('classifier__learning_rate', 0.31167512125611246), ('classifier__max_depth', 12), ('classifier__min_child_weight', 2), ('classifier__n_estimators', 182), ('classifier__reg_alpha', 0.44024813586192624), ('classifier__reg_lambda', 3.0385010957535226), ('classifier__scale_pos_weight', 0.7983193277310925)])
Train f1:  1.0
Mean f1 cross-validated:  0.9709521234025283
		 precision 		 recall 		 f1-score
0 		 0.98 			 0.96
1 		 0.97 			 0.98
Accuracy 							 0.97
macro avg 	 0.97 			 0.97 			 0.97

Test f1 score:  0.8719745643372645
              precision    recall  f1-score   support

       False       0.83      0.96      0.89       228
        True       0.95      0.77      0.85       203

    accuracy                           0.87       431
   macro avg       0.89      0.87      0.87       431
weighted avg       0.89      0.87      0.87       431
 


